Remote control Jetbot using Virtual gamepad
======


In [ ]:
# This cell should only run once.
import os
# set the current working directory. This is required by isaac.
os.chdir("../..")
os.getcwd()


In [ ]:
simulation =True
from packages.pyalice import Application, Message, Codelet

# Creates an empty Isaac application
app = Application(name="jetbot_application")

The Robot Engine Bridge enables communication between Omniverse and the Isaac SDK. When a REB application is created, a simulation-side Isaac SDK application is started, allowing messages to be sent to, or published from, the REB Components. The simulation Subgraph is loaded into our Isaac application, allowing the exchange of messages between our application and the REB Components present in the Omniverse model over TCP. Thus, by loading in the simulation subgraph and using the Camera and Differential Base components, our Isaac application can receive the image stream from Omniverse’s Viewport and can transmit commands to be effectuated in simulation. 

In [ ]:
if simulation:
    # Loads the simulation_tcp subgraph into the Isaac application, adding all nodes, components,
    # edges, and configurations
    app.load(filename="apps/jetbot/simulation_tcp.subgraph.json", prefix="simulation")

    # Gets a reference to the interface node of the subgraph having a prefix of "simulation"
    simulation_node = app.nodes["simulation.interface"]
    

The Robot Remote Control component can send commands to the Differential Base to control the Jetbot model. Therefore, we add a node to which a component of type RobotRemoteControl is added. Nodes can be thought of as a container to group related components in an Isaac application. As the RobotRemoteControl component will be used to generate commands in the form of the desired state of a Segway, an edge is added between the “Segway_cmd” channel of the RobotRemoteControl component and the “base_command” channel of the simulation subgraph. This allows the REB Differential Base in Omniverse to receive the desired Segway states and move the Jetbot model in accordance with the received command. 

In [ ]:
if simulation:
    # Creating a new node in the Isaac application named "robot_remote"
    robot_remote_node = app.add("robot_remote")

    # Loads the navigation module, allowing components requiring this module to be added to the application
    app.load_module("navigation")

    # Add the RobotRemoteControl and FailsafeHeartbeat components to the robot_remote node
    robot_remote_control_component = robot_remote_node.add(name="RobotRemoteControl",                  
        ctype=app.registry.isaac.navigation.RobotRemoteControl)

    failsafe_component = robot_remote_node.add(name="FailsafeHeartbeat",                  
        ctype=app.registry.isaac.alice.FailsafeHeartbeat)

    # Set component configuration parameters
    robot_remote_control_component.config["tick_period"] = "10ms"
    failsafe_component.config["heartbeat_name"] = "deadman_switch"
    failsafe_component.config["failsafe_name"] = "robot_failsafe"
    failsafe_component.config["interval"] = 0.25

    # Makes dataflow connection between "segway_cmd" channel of RobotRemoteControl component, and the 
    # "base_command" channel of the REB Differental Base in simulation
    app.connect(robot_remote_control_component, "segway_cmd", simulation_node["input"], "base_command")


To generate a corresponding command, the Robot Remote Control component must receive either JoystickStateProto messages from its "js_state" channel, or messages consisting of a linear and angular velocity over its "ctrl" channel. Here, we add the virtual gamepad subgraph which can be used to generate "JoystickStateProto" messages required by the Robot Remote Control component. Therefore, we establish the necessary connection.

In [ ]:
if simulation:
    # Loads the virtual_gamepad subgraph into the application
    app.load(filename="apps/jetbot/virtual_gamepad.subgraph.json", prefix="virtual_gamepad")

    # Finds a reference to the component named "interface", located in the subgraph node of the virtual gamepad 
    # subgraph. The component named "interface" is of type Subgraph, meaning all messages coming to or from the
    # virtual gamepad subgraph will pass through the channels of the subgraph component. 
    virtual_gamepad_interface = app.nodes["virtual_gamepad.subgraph"]["interface"]

    # Pass messages generated by virtual gamepad to RobotRemoteControl component. 
    app.connect(virtual_gamepad_interface, "joystick", robot_remote_control_component, "js_state")


The virtual gamepad widget in Sight allows us to use the WASD keys of the keyboard to steer the Jetbot in simulation, thus making the connection between Isaac and Omniverse more concrete. Prior to starting the Isaac application, the REB application is created by opening the jetbot.usd file in Omniverse, navigating to the Robot Engine Bridge extension and clicking "create application", followed by pressing the "Play" button. The Isaac application can now be started by executing the following piece of code. 

In [ ]:
app.start()

Open Sight by going to (Your-IP-Address):3000 in your browser (or localhost:3000 if the Isaac SDK is running on your local machine), and control the Jetbot in simulation with the Virtual Gamepad as shown below. 

In [ ]:
app.stop()

Running Inference in Simulation 
======
Please note the following section requires a training environment built in simulation and a model trained.

With the simulation environment and a trained model (the .etlt file generated by following the Object Detection with DetectNetv2 pipeline), we can run inference using data streamed from simulation using the detectnet subgraph. The subgraph receives ImageViewer proto messages from its "image" channel, performs inference on the received images, and transmits a Detections2Proto message containing the bounding box position, label, and confidence for each of the detections. Upon loading the subgraph, configuration parameters are adjusted according to how training was conducted using the object detection pipeline. 

In [ ]:
app.load(filename="packages/detect_net/apps/detect_net_inference.subgraph.json", prefix="detect_net")

# Setting configuration parameters of components used in the detect-net subgraph to allow the trained
# model to be used, and training parameters specified.
inference_component = app.nodes["detect_net.tensor_r_t_inference"]["isaac.ml.TensorRTInference"]
inference_component.config.model_file_path = "external/jetbot_ball_detection_resnet_model/jetbot_ball_detection_resnet18.etlt"
inference_component.config.etlt_password = "nvidia"

decoder_component = app.nodes["detect_net.detection_decoder"]["isaac.detect_net.DetectNetDecoder"]
decoder_component.config.labels = ["sphere"]

# Changing Detectnet Subgraph to accommodate Omniverse viewport (720 x 1280) and
# dimensions used to train model
if simulation:
    inference_component.config["input_tensor_info"] = [
            {
                "operation_name": "input_1",
                "channel": "image",
                "dims": [3, 368, 640],
                "uff_input_order": "channels_last"
            }
        ]
    decoder_component.config["output_scale"] = [720, 1280]
    encoder_component = app.nodes["detect_net.tensor_encoder"]["isaac.ml.ColorCameraEncoderCuda"]
    encoder_component.config["rows"] = 368



With the subgraph loaded and configuration parameters set, we can relay Omniverse's viewport stream, captured by the REB Camera, to the detectnet subgraph, allowing inference to be performed on simulation data.

In [ ]:
detect_net_interface = app.nodes["detect_net.subgraph"]["interface"]

if simulation:
    # Allows image stream from Omniverse to flow to detect-net 
    app.connect(simulation_node["output"], "color", detect_net_interface, "image")


In [ ]:
app.start()

Upon opening the jetbot_inference.usd file in Omniverse, creating the Robot Engine Bridge application, and starting both the simulation and he Isaac application, the performance of the detection model can be verified. 

In [ ]:
app.stop()

Jetbot Autonomously Following Objects in Simulation
======
Now that objects are being correctly detected in simulation, we need to implement the control logic to move the Jetbot model such that it keeps the desired object both just in front of it and horizontally centered. To accomplish this, we first define a couple helper functions to parse detections2proto messages, determine the pixel coordinates of the center of a bounding box, determine the area (in pixels) of a bounding box, and find the detection of a specified label whose bounding box center is closest to the target location. Bounding box area will later be used to estimate how close or far a detected object is from the Jetbot. 

In [ ]:
import numpy as np
import json

def get_parsed_detections(detections_msg):
    """Parses and reformats Detections2Proto messages"""
    detections_msg_json = detections_msg.json
    zipped = zip(detections_msg_json['predictions'], detections_msg_json['boundingBoxes'])
    zipped_list = list(zipped)
    
    detections = {}
    for k, sublist in enumerate(zipped_list):
        for sub_sublist in sublist:
            # If key already present; append to the existing dict.
            if (k in detections):
                detections[k] = {**detections[k],**sub_sublist}
            # If key not present insert the first attribute of the detected object.
            else:
                detections[k] = {**sub_sublist}
    return detections

def norm(vec, target):
    """Computes the length of the 2D vector"""
    return np.sqrt((vec[0]-target[0])**2 + (vec[1]-target[1])**2)

def get_detection_center(detection):
    """Computes the center x, y coordinates of the object x = rows of image; y = cols of image """
    center_x = (detection['min']['x'] + detection['max']['x']) / 2.0 - 0.5
    center_y = (detection['min']['y'] + detection['max']['y']) / 2.0 - 0.5
    return (center_x, center_y)

def get_detection_area(detection):
    """Computes the area (in pixels) of a detection"""
    detection_width = detection['max']['x'] - detection['min']['x']
    detection_height = detection['max']['y'] - detection['min']['y']
    detection_area = detection_width * detection_height
    return detection_area

def find_closest_matching_detection(detections_dict, target, label):
    """Finds the closest detection to target pixel location in detections_dict, having the specified label"""
    closest_matching_detection = None
    closest_matching_detection_dist = np.inf
    
    for detection in detections_dict.values():

        if detection["label"] == label:
            detection_center = get_detection_center(detection)
            detection_dist = norm(detection_center, target)

            if detection_dist < closest_matching_detection_dist:
                closest_matching_detection = detection
                closest_matching_detection_dist = detection_dist
    
    return closest_matching_detection

    


With the helper functions in place and detections being made in simulation, we can develop a Python Codelet to control the Jetbot, which will use detections messages to compute the desired motor commands of the Jetbot, and then publish messages containing these commands. Codelets are functionally equivalent to the built-in COMPONENTS provided by the Isaac SDK in the sense that they both send and receive messages, however, Codelets provide a way for us to create a custom component so we can execute user-defined code within our Isaac application. You can learn more about creating Codelets HERE. 

As the real Jetbot’s motors are controlled with Pulse Width Modulator (PWM) duty cycle commands, the created Codelet will generate messages containing commands of this type. Linear speed of the Jetbot is determined based on the area of the bounding box of a detected object; If the area of the detected object is too small, the Jetbot will move closer to the object, whereas if the area is too large, the Jetbot will back away. Similarly, angular speed is set based on the horizontal offset of the detection from the center of the Jetbot’s view. Finally, motor commands are calculated based on the linear and angular speed, and messages containing the commands are published. 


In [ ]:
# Generates PWM commands to follow desired object
class JetbotControl(Codelet):
    
    def start(self):
        self.rx = self.isaac_proto_rx("Detections2Proto", "detections")
        self.tx = self.isaac_proto_tx("StateProto", "motor_command")

        # Ticks when new detections message is received
        self.tick_on_message(self.rx)

    def tick(self):
        # Receives a Detections2Proto message
        rx_message = self.rx.message

        # Reads configuration parameters set outside of Codelet
        label = self.config.label
        image_width = self.config.image_width
        image_height = self.config.image_height
        min_pwm = self.config.min_pwm  # Smallest motor command required to move real Jetbot
        angular_gain = self.config.angular_gain
        target_coverage = self.config.target_coverage

        parsed_detections = get_parsed_detections(rx_message)

        image_horizontal_center = image_width / 2.0
        image_vertical_center = image_height / 2.0
        image_center =  [image_vertical_center, image_horizontal_center]

        detection = find_closest_matching_detection(parsed_detections, image_center, label)

        if detection is None:  
            # Do not move if there isn't a detection with matching label
            left_motor_command = 0.0
            right_motor_command = 0.0
        else:  
            # Generate PWM commands to move towards detection by keeping the detection horizontally centered, 
            # and the fraction of the image the bounding box covers equal to target_coverage
            
            # Compute areas
            image_area = image_width * image_height
            target_area = target_coverage * image_area
            detection_area = get_detection_area(detection)
            
            # Use areas to determine linear speed
            # min_pwm is used here to eliminate dead zones
            if detection_area < target_area:
                linear_speed = min_pwm + (1 - min_pwm)*(target_area - detection_area) / target_area
            else:
                linear_speed = -min_pwm + (1 - min_pwm)*(target_area - detection_area) / (image_area - target_area)
            
            # Use horizontal offset of detection from image center to determine angular speed
            detection_center = get_detection_center(detection)
            angular_speed = (image_horizontal_center - detection_center[1]) / image_horizontal_center

            # Computes motor commands based on desired linear and angular speeds, ensuring PWM commands are in Jetbot's
            # acceptable range of [-1, 1]
            min_motor_command = -1
            max_motor_command = 1
            left_motor_command = float(np.clip(linear_speed - angular_gain * angular_speed, min_motor_command, max_motor_command))
            right_motor_command = float(np.clip(linear_speed + angular_gain * angular_speed, min_motor_command, max_motor_command))

        # Initializes, populates, and transmits a StateProto message containing motor commands
        tx_message = self.tx.init()
        data = tx_message.proto.init('data', 2)

        data[0] = left_motor_command
        data[1] = right_motor_command
        
        self.tx.publish()



The created Codelet must be added to the Isaac application, just like a normal component. The configuration parameters are then set and an edge added between the Codelet and the detect-net subgraph so that the detections messages can be used to generate control commands.

In [ ]:
# Create a new node, and add the JetbotControl Codelet to the node.
controller_node = app.add("controller")
jetbot_control_component = controller_node.add(JetbotControl)

# Set the configuration parameters of the JetbotControl Codelet
if simulation:
    jetbot_control_component.config.image_width = 1280
    jetbot_control_component.config.image_height = 720
else:
    jetbot_control_component.config.image_width = 640
    jetbot_control_component.config.image_height = 360
jetbot_control_component.config.label = "sphere"
jetbot_control_component.config.target_coverage = 0.05
jetbot_control_component.config.angular_gain = 0.057
jetbot_control_component.config.min_pwm = 0.25

# Pass detections to JetbotControl Codelet
app.connect(detect_net_interface, "detections", jetbot_control_component, "detections")



While the added Codelet can generate motor commands compatible with the real Jetbot, in simulation the Jetbot is controlled by sending Segway commands to the REB Differential Base. Segway commands can be generated by providing linear and angular velocities to the “ctrl” channel of the previously created RobotRemoteControl component. To convert PWM commands generated by our controller into the linear and angular commands needed, a relationship between motor commands sent to the Jetbot and the speed at which the real Jetbot travels must be established. The mapping between motor command and velocity was found by experimentally measuring the time taken for the real Jetbot to travel 3 meters. 

In [ ]:
def pwm_to_velocity(pwm_command, min_pwm_command):
    """Computes velocity (in [m/s]) of real Jetbot when both motors are set to "pwm_command" based on experimental data"""
    command_abs = np.abs(pwm_command)

    # min_pwm_command represents the interval of commands sent to the jetbot which do not cause movement:
    # [-min_pwm_command, min_pwm_command]
    if command_abs < min_pwm_command:
        velocity = 0
    else:
        velocity = float(np.sign(pwm_command) * (2.0328 * command_abs - 0.0948))
    
    return velocity


A second Codelet can now be created to adapt the PWM commands so that they are able to be used in simulation. With the help of our recently defined “pwm_to_velocity” function, the velocity of each wheel can be calculated. Then, using the dynamics equations of a differential base, linear and angular velocity can be calculated from the wheel velocities. 

In [ ]:
# Converts PWM commands into linear and angular velocities
class SimulationAdapter(Codelet):

    def start(self):

        self.rx = self.isaac_proto_rx("StateProto", "motor_command")
        self.tx = self.isaac_proto_tx("StateProto", "velocity_command")
        self.tick_on_message(self.rx)

    def tick(self):

        rx_message = self.rx.message

        min_pwm_command = self.config.min_pwm_command
        simulation_linear_gain = self.config.simulation_linear_gain
        simulation_angular_gain = self.config.simulation_angular_gain

        data = rx_message.json['data']
        
        left_motor_command = data[0]
        right_motor_command = data[1]

        left_wheel_velocity = pwm_to_velocity(left_motor_command, min_pwm_command)
        right_wheel_velocity = pwm_to_velocity(right_motor_command, min_pwm_command)

        # Distance between wheels of Jetbot [m]
        length = 0.1143

        # Linear and angular velocity resulting from PWM command
        linear_velocity = (left_wheel_velocity + right_wheel_velocity) / 2.0
        angular_velocity = (left_wheel_velocity - right_wheel_velocity) / length

        # Gains were found using a REB RigidBodySink and measuring velocity traveled in simulation,
        # versus linear and angular command sent to simulation.  
        simulation_linear_command = simulation_linear_gain * linear_velocity
        simulation_angular_command = simulation_angular_gain * angular_velocity

        # Initializes, populates, and publishes commands containing linear and angular velocities
        tx_message = self.tx.init()
        data = tx_message.proto.init('data', 2)

        data[0] = simulation_linear_command
        data[1] = simulation_angular_command
        
        self.tx.publish()


With the Simulation Adapter Codelet defined, we may now add it to our Isaac application. An edge is added from the Jetbot Control Codelet to the Simulation Adapter, allowing the Adapter to receive PWM commands from the controller. Once the commands are converted into linear and angular velocities, they must be sent to the RobotRemoteControl component as previously discussed, so we need to add the corresponding edge. 

In [ ]:
if simulation:
    # Create a new node, and add the SimulationAdapter Codelet to the node.
    adapter_node = app.add("adapter")
    simulation_adapter_component = adapter_node.add(SimulationAdapter)

    # Set the configuration parameters of the SimulationAdapter Codelet
    simulation_adapter_component.config.min_pwm_command = 0.2
    simulation_adapter_component.config.simulation_linear_gain = 0.27
    simulation_adapter_component.config.simulation_angular_gain = -0.16

    # Pass motor commands calculated by the JetbotControl Codelet to the SimulationAdapter Codelet
    app.connect(jetbot_control_component, "motor_command", simulation_adapter_component, "motor_command")

    # Pass linear and angular velocity commands from the SimulationAdapter Codelet to the RobotRemoteControl component.
    app.connect(simulation_adapter_component, "velocity_command", robot_remote_control_component, "ctrl")



Now we’re ready to autonomously follow a ball in simulation. Upon opening the jetbot_follow.usd file in Omniverse, create the Robot Engine Bridge application, and start the Isaac application by running the next cell.

In [ ]:
app.start()

You'll notice that despite the Jetbot detecting objects, it isn't moving. The reason is that the Robot Remote Control component will only send commands while the deadman switch is pressed for safety reasons. But there aren't any safety concerns in simulation! Lets go ahead and disable that.

In [ ]:
 if simulation:
    robot_remote_control_component.config["disable_deadman_switch"] = True

You should now see your Jetbot following balls as they appear before it in simulation. Cool! Tweak the config parameters of the Wheel Velocity Control Generator Codelet to your likening, and let's finish bridging the gap between simulation and reality!

In [ ]:
app.stop()